In [1]:
import csv

In [2]:
class User: # пользователь
    name = ''
    movie_rates = []

In [3]:
class Movie_rate:  # фильм
    def __init__(self, name, rate):
        self.name = name
        self.rate = rate

In [15]:
def parse_movie_names(row):
    row = list(map(lambda name: name.strip(), row))
    return list(filter(lambda name: name != '', row))

In [20]:
def load_users(filename):
    data_file = open(filename)
    reader = csv.reader(data_file)
    
    movie_names = parse_movie_names(next(reader))
    users = []
    
    for row in reader:
        user = User()
        user.name = row[0]
        user.movie_rates = list(
            map(lambda movie_name, movie_rate: Movie_rate(movie_name, int(movie_rate)), movie_names, row[1:])
        )
        
        users.append(user)
        
    return users

In [21]:
users = load_users('./data.csv')
for user in users:
    print(user.name)
    for movie_rate in user.movie_rates:
        print(movie_rate.name, movie_rate.rate)

User 1
Movie 1 -1
Movie 2 4
Movie 3 1
Movie 4 5
Movie 5 -1
Movie 6 -1
Movie 7 -1
Movie 8 1
Movie 9 3
Movie 10 -1
Movie 11 5
Movie 12 5
Movie 13 1
Movie 14 3
Movie 15 3
Movie 16 3
Movie 17 -1
Movie 18 4
Movie 19 2
Movie 20 2
Movie 21 2
Movie 22 4
Movie 23 -1
Movie 24 -1
Movie 25 3
Movie 26 4
Movie 27 -1
Movie 28 2
Movie 29 4
Movie 30 1
User 2
Movie 1 4
Movie 2 5
Movie 3 2
Movie 4 3
Movie 5 3
Movie 6 4
Movie 7 -1
Movie 8 5
Movie 9 3
Movie 10 2
Movie 11 2
Movie 12 -1
Movie 13 5
Movie 14 5
Movie 15 4
Movie 16 1
Movie 17 -1
Movie 18 2
Movie 19 4
Movie 20 5
Movie 21 -1
Movie 22 2
Movie 23 -1
Movie 24 3
Movie 25 1
Movie 26 5
Movie 27 -1
Movie 28 5
Movie 29 2
Movie 30 2
User 3
Movie 1 4
Movie 2 2
Movie 3 5
Movie 4 4
Movie 5 1
Movie 6 5
Movie 7 -1
Movie 8 3
Movie 9 5
Movie 10 2
Movie 11 1
Movie 12 -1
Movie 13 1
Movie 14 1
Movie 15 1
Movie 16 1
Movie 17 3
Movie 18 3
Movie 19 3
Movie 20 -1
Movie 21 4
Movie 22 -1
Movie 23 5
Movie 24 4
Movie 25 5
Movie 26 1
Movie 27 2
Movie 28 1
Movie 29 -1
Movie 3

User 34
Movie 1 -1
Movie 2 5
Movie 3 -1
Movie 4 -1
Movie 5 1
Movie 6 -1
Movie 7 5
Movie 8 1
Movie 9 1
Movie 10 5
Movie 11 3
Movie 12 2
Movie 13 2
Movie 14 2
Movie 15 5
Movie 16 3
Movie 17 4
Movie 18 3
Movie 19 -1
Movie 20 3
Movie 21 3
Movie 22 3
Movie 23 4
Movie 24 -1
Movie 25 3
Movie 26 1
Movie 27 5
Movie 28 2
Movie 29 2
Movie 30 5
User 35
Movie 1 4
Movie 2 -1
Movie 3 5
Movie 4 4
Movie 5 2
Movie 6 2
Movie 7 3
Movie 8 1
Movie 9 1
Movie 10 5
Movie 11 -1
Movie 12 2
Movie 13 5
Movie 14 4
Movie 15 -1
Movie 16 2
Movie 17 -1
Movie 18 3
Movie 19 -1
Movie 20 -1
Movie 21 5
Movie 22 1
Movie 23 2
Movie 24 1
Movie 25 5
Movie 26 5
Movie 27 1
Movie 28 -1
Movie 29 5
Movie 30 5
User 36
Movie 1 3
Movie 2 2
Movie 3 -1
Movie 4 5
Movie 5 5
Movie 6 1
Movie 7 5
Movie 8 2
Movie 9 1
Movie 10 5
Movie 11 2
Movie 12 4
Movie 13 3
Movie 14 4
Movie 15 3
Movie 16 -1
Movie 17 -1
Movie 18 -1
Movie 19 2
Movie 20 -1
Movie 21 -1
Movie 22 2
Movie 23 4
Movie 24 3
Movie 25 4
Movie 26 5
Movie 27 2
Movie 28 1
Movie 29 3
Movie